### Converting the txt to DataFrame

In [4]:
import pandas as pd

# Step 1: Define the file path
file_name = "SLC19A2.txt"

# Step 3: Parse the file into a DataFrame (assuming it's tab-separated)
df = pd.read_csv(file_name, sep="\t")
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

# Step 4: Save the DataFrame as a CSV file
csv_file_name = "csv_files/parsed_file.csv"
df.to_csv(csv_file_name, index=False)
print(f"\nData saved as '{csv_file_name}'.")




First 5 rows of the DataFrame:
  #chr        pos variation variant_type   snp_id clinical_significance  \
0    1  169476945   A>C,G,T          snv  1983546                benign   
1    1  169486197     G>C,T          snv  2056926                benign   
2    1  169485770     G>A,C          snv  2072757                benign   
3    1  169465789     C>A,T          snv  2678166                benign   
4    1  169468787       A>G          snv  3737682  likely-benign;benign   

                 validation_status  \
0  by-frequency;by-alfa;by-cluster   
1  by-frequency;by-alfa;by-cluster   
2  by-frequency;by-alfa;by-cluster   
3  by-frequency;by-alfa;by-cluster   
4  by-frequency;by-alfa;by-cluster   

                                     function_class     gene  \
0                                    intron_variant  SLC19A2   
1  upstream_transcript_variant;2KB_upstream_variant  SLC19A2   
2                               5_prime_UTR_variant  SLC19A2   
3                               

In [5]:

# Remove rows that repeat the column header.
# We'll assume that any row where the first column's value equals the header name should be dropped.
header_name = df.columns[0]
df_clean = df[df[header_name] != header_name].copy()

print("\nFirst 5 rows after cleaning:")
print(df_clean.head())

# Save the cleaned DataFrame as a CSV file
csv_file_name = "csv_files/parsed_file_cleaned.csv"
df_clean.to_csv(csv_file_name, index=False)
print(f"\nCleaned data saved as '{csv_file_name}'.")




First 5 rows after cleaning:
  #chr        pos variation variant_type   snp_id clinical_significance  \
0    1  169476945   A>C,G,T          snv  1983546                benign   
1    1  169486197     G>C,T          snv  2056926                benign   
2    1  169485770     G>A,C          snv  2072757                benign   
3    1  169465789     C>A,T          snv  2678166                benign   
4    1  169468787       A>G          snv  3737682  likely-benign;benign   

                 validation_status  \
0  by-frequency;by-alfa;by-cluster   
1  by-frequency;by-alfa;by-cluster   
2  by-frequency;by-alfa;by-cluster   
3  by-frequency;by-alfa;by-cluster   
4  by-frequency;by-alfa;by-cluster   

                                     function_class     gene  \
0                                    intron_variant  SLC19A2   
1  upstream_transcript_variant;2KB_upstream_variant  SLC19A2   
2                               5_prime_UTR_variant  SLC19A2   
3                               3_

In [6]:
df_var = df_clean.copy()

# The variation format is like 'A>G', so we split at '>' and take the first part
df_var['original'] = df_var['variation'].str.split('>', expand=True)[0]
# Get the mutant part (after '>')
df_var['mutant'] = df_var['variation'].str.split('>', expand=True)[1]

df_var.shape


(9644, 12)

In [9]:
# Separate rows with multiple mutants (e.g., G>A,C)
expanded_df = []

for index, row in df_var.iterrows():
    # Check if the mutant column contains a comma (multiple mutations)
    if ',' in str(row['mutant']):
        # Split the mutant values by comma
        mutants = row['mutant'].split(',')
        
        # Create a new row for each mutant
        for mutant in mutants:
            new_row = row.copy()
            new_row['mutant'] = mutant
            expanded_df.append(new_row)
    else:
        # If there's only one mutant, keep the row as is
        expanded_df.append(row)

# Create a new DataFrame from the expanded list
expanded_df = pd.DataFrame(expanded_df)

# Reset the index of the new DataFrame
expanded_df.reset_index(drop=True, inplace=True)

# Display the first few rows to verify the expansion
expanded_df.shape

# Export the expanded DataFrame to a CSV file
expanded_df.to_csv('csv_files/expanded_variants.csv', index=False)
print(f"Exported {len(expanded_df)} rows to expanded_variants.csv")


Exported 12647 rows to expanded_variants.csv


In [10]:
# Create a DataFrame with normal versions for each unique SNP ID
normal_variants = []

# Get unique SNP IDs
unique_snp_ids = expanded_df['snp_id'].unique()

for snp_id in unique_snp_ids:
    # Get the first row for this SNP ID as a template
    template_row = expanded_df[expanded_df['snp_id'] == snp_id].iloc[0].copy()
    
    # Create a normal version of this variant
    normal_row = template_row.copy()
    normal_row['clinical_significance'] = 'normal'
    normal_row['mutant'] = normal_row['original']  # Set mutant to be the same as original
    
    # Add to our list of normal variants
    normal_variants.append(normal_row)

# Create a DataFrame from the normal variants
normal_df = pd.DataFrame(normal_variants)

# Combine the expanded DataFrame with the normal variants
combined_df = pd.concat([expanded_df, normal_df], ignore_index=True)

# Reset the index
combined_df.reset_index(drop=True, inplace=True)

# Display the shape of the combined DataFrame
print(f"Original expanded DataFrame shape: {expanded_df.shape}")
print(f"Normal variants added: {len(normal_variants)}")
print(f"Combined DataFrame shape: {combined_df.shape}")

# Export the combined DataFrame to a CSV file
combined_df.to_csv('combined_variants_with_normal.csv', index=False)
print(f"Exported {len(combined_df)} rows to combined_variants_with_normal.csv")


Original expanded DataFrame shape: (12647, 12)
Normal variants added: 9486
Combined DataFrame shape: (22133, 12)
Exported 22133 rows to combined_variants_with_normal.csv


In [11]:
# Sort the combined DataFrame by snp_id to group variants with the same SNP ID together
combined_df = combined_df.sort_values(by='snp_id')

# Reset the index after sorting
combined_df.reset_index(drop=True, inplace=True)

# Export the sorted DataFrame to a CSV file
combined_df.to_csv('combined_variants_sorted_by_snp_id.csv', index=False)
print(f"Exported {len(combined_df)} rows to combined_variants_sorted_by_snp_id.csv")

# Display the first few rows to verify sorting
print("\nFirst few rows of the sorted DataFrame:")
print(combined_df.head())


Exported 22133 rows to combined_variants_sorted_by_snp_id.csv

First few rows of the sorted DataFrame:
  #chr        pos variation variant_type      snp_id clinical_significance  \
0    1  169476354       T>C          snv  1000230073                   NaN   
1    1  169476354       T>C          snv  1000230073                normal   
2    1  169475878       T>C          snv  1000304891                normal   
3    1  169475878       T>C          snv  1000304891                   NaN   
4    1  169483460       T>C          snv  1000361639                normal   

                 validation_status  function_class     gene  \
0  by-frequency;by-alfa;by-cluster  intron_variant  SLC19A2   
1  by-frequency;by-alfa;by-cluster  intron_variant  SLC19A2   
2  by-frequency;by-alfa;by-cluster  intron_variant  SLC19A2   
3  by-frequency;by-alfa;by-cluster  intron_variant  SLC19A2   
4             by-frequency;by-alfa  intron_variant  SLC19A2   

                                           freque

### Correcting SNPs

In [26]:
from Bio import Entrez, SeqIO
import re
import time
import io


print("--- Step 1: Setup and Load Data ---")

# --- Configuration ---
Entrez.email = "youssef.nageh@gmail.com" # ALWAYS provide your email
Entrez.api_key = "a8e71186e89e68cd57485f79cb0afe125008"  # Optional, but recommended
entrez_delay = 0.2 # Delay between NCBI calls

# --- Global Variables ---
chromosome_acc = "NC_000001.11"
""" 
genomic_region_start = 227683763 - 1000 # this is for SLC19A3
genomic_region_end = 227718028 + 2000
"""
genomic_region_start = 169463909 - 2000 #slc19a2
genomic_region_end = 169485970 + 2000

print(f"--- Fetching WT Genomic DNA Region ---")
print(f"Fetching {chromosome_acc} from {genomic_region_start} to {genomic_region_end}...")
wt_genomic_sequence_region = None
try:
    time.sleep(entrez_delay)
    handle_dna = Entrez.efetch(db="nucleotide",
                               id=chromosome_acc,
                               rettype="fasta",
                               retmode="text",
                               seq_start=genomic_region_start,
                               seq_stop=genomic_region_end,
                               strand=1)
    dna_record = SeqIO.read(io.StringIO(handle_dna.read()), "fasta")
    handle_dna.close()
    wt_genomic_sequence_region = str(dna_record.seq).upper()
    print(f"Successfully fetched WT genomic region sequence (Length: {len(wt_genomic_sequence_region)}).")
    expected_len = genomic_region_end - genomic_region_start + 1
    if len(wt_genomic_sequence_region) != expected_len:
        print(f"WARNING: Fetched genomic sequence length ({len(wt_genomic_sequence_region)}) does not match expected length ({expected_len}).")
except Exception as e:
    print(f"ERROR fetching WT genomic DNA: {e}")
    print("Cannot proceed without WT genomic sequence.")
    wt_genomic_sequence_region = None

def get_snp_api_text(snp_id_numeric):
    """
    Fetches SNP info text (requesting XML format) from NCBI Efetch.

    Args:
        snp_id_numeric (str): The numeric part of the rsID.

    Returns:
        str: Raw text content from API, or None on failure.
    """
    print(f"      API Call: Fetching text for rs{snp_id_numeric}")
    raw_text_output = None
    if not snp_id_numeric or not snp_id_numeric.isdigit():
         print(f"      API Call ERROR: Invalid numeric SNP ID: '{snp_id_numeric}'")
         return None
    try:
        time.sleep(entrez_delay) # Respect NCBI rate limits
        handle = Entrez.efetch(db="snp", id=snp_id_numeric, rettype="xml", retmode="xml")
        output_bytes = handle.read()
        handle.close()
        if output_bytes:
            raw_text_output = output_bytes.decode('utf-8')
            print(f"      API Call SUCCESS: Fetched text (length {len(raw_text_output)}).")
        else:
             print(f"      API Call WARNING: No data received for rs{snp_id_numeric}.")
    except Entrez.HTTPError as http_err:
         print(f"      API Call NETWORK ERROR for rs{snp_id_numeric}: {http_err}")
    except Exception as e:
         print(f"      API Call GENERAL ERROR for rs{snp_id_numeric}: {e}")
    return raw_text_output

--- Step 1: Setup and Load Data ---
--- Fetching WT Genomic DNA Region ---
Fetching NC_000001.11 from 169461909 to 169487970...
Successfully fetched WT genomic region sequence (Length: 26062).


In [27]:
def parse_spdi_alleles_from_text(api_text, chromosome_acc):
    """
    Parse SPDI allele notation from NCBI SNP API XML response.
    
    Args:
        api_text (str): Raw XML text from NCBI SNP API
        chromosome_acc (str): Chromosome accession to match (e.g., 'NC_000001.11')
    
    Returns:
        tuple: (reference_allele, alternate_allele) or (None, None) if parsing fails
    """
    if not api_text or not chromosome_acc:
        print("      SPDI Parse ERROR: Missing API text or chromosome accession")
        return None, None
    
    try:
        # Look for SPDI pattern in the XML text
        # Format: sequence_id:position:deleted_sequence:inserted_sequence
        import re
        
        # Pattern to match SPDI notation with the specific chromosome accession
        pattern = rf'{chromosome_acc}:(\d+):([ACGT]*):([ACGT]*)'
        
        # Find all matches
        matches = re.findall(pattern, api_text)
        
        if not matches:
            print(f"      SPDI Parse WARNING: No SPDI notation found for {chromosome_acc}")
            return None, None
        
        # Use the first match (could be refined to select the canonical one if needed)
        position, ref_allele, alt_allele = matches[0]
        
        # Validate alleles
        if ref_allele == '' and alt_allele == '':
            print("      SPDI Parse ERROR: Both reference and alternate alleles are empty")
            return None, None
            
        print(f"      SPDI Parse SUCCESS: Found ref='{ref_allele}', alt='{alt_allele}'")
        return ref_allele, alt_allele
        
    except Exception as e:
        print(f"      SPDI Parse ERROR: {e}")
        return None, None

# --- [PREVIOUS CODE: Imports, Setup, WT Fetch, Helper Functions] ---
# --- [PLACEHOLDER: Ensure combined_df is loaded] ---
# --- [Check if required variables exist] ---

# --- Validation and Correction Loop ---
corrections_made = 0
validation_failures = 0 # Counter for persistent/uncorrected errors in non-skipped rows
skipped_insertions = 0 # Counter for rows skipped due to original == '-'

# Ensure required columns are present in combined_df
required_cols = ['pos', 'original', 'mutant', 'snp_id', 'clinical_significance']
if not all(col in combined_df.columns for col in required_cols):
    print(f"ERROR: DataFrame 'combined_df' missing one or more required columns: {required_cols}")
    # Optionally exit or handle error
else:
    # Filter rows where clinical_significance is not 'normal'
    target_mask = ~combined_df['clinical_significance'].astype(str).str.lower().eq('normal')
    print(f"Processing {target_mask.sum()} rows where clinical_significance is not 'normal'.")

    # Use indices for direct update with .loc on combined_df
    for index in combined_df[target_mask].index:
        row = combined_df.loc[index] # Get row data

        try:
            # Extract original allele first to check for '-'
            original_allele = str(row['original']).upper() if pd.notna(row['original']) else ''
            # Still extract pos and snp_id for potential logging later, even if skipping
            pos = int(row['pos'])
            snp_id = str(row['snp_id']) if pd.notna(row['snp_id']) else ''

        except (ValueError, TypeError) as e:
             # If basic info can't be read, skip and maybe count as failure
             print(f"  SKIP (Index {index}): Error converting basic row data (pos/original/snp_id): {e}.")
             # validation_failures += 1 # Decide if this counts as failure
             continue # Skip this row

        # --- *** ADDED: Skip rows where original allele is '-' *** ---
        if original_allele == '-':
            skipped_insertions += 1
            # Optional: print a message if you want to see which rows are skipped
            # print(f"  SKIP (snp {snp_id}, pos {pos}): Row skipped because original allele is '-'.")
            continue # Immediately go to the next iteration of the loop
        # --- *** END OF ADDED SKIP LOGIC *** ---

        # --- If we are here, original_allele is NOT '-' ---
        # Now extract mutant allele if needed for later steps
        try:
            mutant_allele = str(row['mutant']).upper() if pd.notna(row['mutant']) else ''
        except (ValueError, TypeError) as e:
            print(f"  SKIP (snp {snp_id}, pos {pos}): Error converting mutant allele: {e}.")
            validation_failures += 1 # Count failure if mutant allele bad for non '-' row
            continue

        # --- Basic Calculation & Bounds Check (only for non '-' rows) ---
        index_0based = pos - genomic_region_start
        ref_len = len(original_allele) # Will not be 0 here unless original was empty string ''

        # Handle potentially empty original allele (if not NaN and not '-')
        if ref_len == 0:
            # print(f"  SKIP (snp {snp_id}, pos {pos}): Empty 'original' allele (and not '-').")
            continue

        # Bounds checks
        if not (0 <= index_0based < len(wt_genomic_sequence_region)):
            print(f"  FAIL (snp {snp_id}, pos {pos}): Position {pos} maps to index {index_0based}, outside fetched WT bounds.")
            validation_failures += 1
            continue
        if index_0based + ref_len > len(wt_genomic_sequence_region):
             print(f"  FAIL (snp {snp_id}, pos {pos}): Original allele '{original_allele}' (len {ref_len}) at index {index_0based} extends beyond fetched WT length.")
             validation_failures += 1
             continue

        # --- Compare (only for non '-' rows) ---
        ref_from_genome = wt_genomic_sequence_region[index_0based : index_0based + ref_len].upper()

        if ref_from_genome != original_allele:
            # --- Mismatch: Attempt Strategy #2 (API Fix) ---
            mismatch_detail = f"Mismatch at pos {pos}: DataFrame 'original'='{original_allele}', Genome='{ref_from_genome}'."
            print(f"  WARN (snp {snp_id}): {mismatch_detail} Trying API Fix...")

            numeric_snp_id = None
            # Extract numeric SNP ID
            if snp_id and isinstance(snp_id, str):
                 if snp_id.lower().startswith('rs') and snp_id[2:].isdigit():
                     numeric_snp_id = snp_id[2:]
                 elif snp_id.isdigit():
                     numeric_snp_id = snp_id

            if not numeric_snp_id:
                print(f"  FAIL (snp {snp_id}): Invalid or missing snp_id ('{snp_id}') for API lookup. Fix attempt failed.")
                validation_failures += 1
                continue

            api_raw_text = get_snp_api_text(numeric_snp_id)

            if api_raw_text:
                spdi_ref, spdi_alt = parse_spdi_alleles_from_text(api_raw_text, chromosome_acc)

                if spdi_ref is not None: # SPDI parsed
                    spdi_ref_len = len(spdi_ref)

                    if spdi_ref_len == 0:
                         print(f"  FAIL (snp {snp_id}): API Fix failed. SPDI Ref allele from API is empty.")
                         validation_failures += 1; continue

                    # Re-check bounds using SPDI ref length
                    if index_0based + spdi_ref_len > len(wt_genomic_sequence_region):
                         print(f"  FAIL (snp {snp_id}): API Fix failed. SPDI Ref '{spdi_ref}' extends beyond fetched WT.")
                         validation_failures += 1; continue

                    # Re-validate WT genome segment vs SPDI Ref
                    expected_ref_spdi = wt_genomic_sequence_region[index_0based : index_0based + spdi_ref_len].upper()

                    if expected_ref_spdi == spdi_ref.upper():
                        print(f"  SUCCESS (snp {snp_id}): API Fix corrected alleles based on SPDI. Updating DataFrame: REF='{spdi_ref}', ALT='{spdi_alt}'.")
                        # Update DataFrame
                        combined_df.loc[index, 'original'] = spdi_ref
                        combined_df.loc[index, 'mutant'] = spdi_alt
                        corrections_made += 1
                    else:
                        print(f"  FAIL (snp {snp_id}): API Fix failed re-check. WT base(s) '{expected_ref_spdi}' != SPDI Ref '{spdi_ref}'. Alleles not corrected.")
                        validation_failures += 1; continue
                else: # SPDI parse failed
                    print(f"  FAIL (snp {snp_id}): API Fix failed (SPDI pattern not found in API response).")
                    validation_failures += 1; continue
            else: # API fetch failed
                print(f"  FAIL (snp {snp_id}): API Fix failed (Error fetching data from API).")
                validation_failures += 1; continue
        # else: Initial match for non '-' row, do nothing

    print(f"\n--- Validation Finished ---")
    print(f"Rows skipped because 'original' allele was '-': {skipped_insertions}")
    print(f"Total Allele Corrections Made via API SPDI lookup (for non '-' rows): {corrections_made}")
    print(f"Total Persistent/Uncorrected Errors (Mismatches or Failures for non '-' rows): {validation_failures}")
    print("\nDataFrame head after validation/correction attempt:")
    # Display relevant columns
    print(combined_df[['#chr', 'pos', 'snp_id', 'original', 'mutant', 'clinical_significance']].head(10))

# --- End of Validation Step ---

Processing 12647 rows where clinical_significance is not 'normal'.

--- Validation Finished ---
Rows skipped because 'original' allele was '-': 391
Total Allele Corrections Made via API SPDI lookup (for non '-' rows): 0
Total Persistent/Uncorrected Errors (Mismatches or Failures for non '-' rows): 0

DataFrame head after validation/correction attempt:
  #chr        pos      snp_id original mutant clinical_significance
0    1  169476354  1000230073        T      C                   NaN
1    1  169476354  1000230073        T      T                normal
2    1  169475878  1000304891        T      T                normal
3    1  169475878  1000304891        T      C                   NaN
4    1  169483460  1000361639        T      T                normal
5    1  169483460  1000361639        T      C                   NaN
6    1  169475969  1000582129        G      G                normal
7    1  169475969  1000582129        G      C                   NaN
8    1  169468609  1000671246     

In [16]:
# Get current timestamp for the filename
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"csv_files/processed_data_{timestamp}.csv"

# Save the dataframe to CSV
combined_df.to_csv(filename, index=False)
print(f"Data saved to {filename}")

# Display the dataframe
combined_df

Data saved to csv_files/processed_data_20250505_145354.csv


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency,original,mutant
0,1,169476354,T>C,snv,1000230073,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,C
1,1,169476354,T>C,snv,1000230073,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,T
2,1,169475878,T>C,snv,1000304891,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,T
3,1,169475878,T>C,snv,1000304891,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,C
4,1,169483460,T>C,snv,1000361639,normal,by-frequency;by-alfa,intron_variant,SLC19A2,C:0.:0:ALFA,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...
22128,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,A
22129,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,C
22130,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,T
22131,1,169481070,C>A,snv,999949244,normal,by-frequency;by-alfa,intron_variant,SLC19A2,A:0.000004:1:TOPMED|A:0.:0:ALFA,C,C


In [17]:
# Filter out rows where 'pos' is not a number
combined_df = combined_df[pd.to_numeric(combined_df['pos'], errors='coerce').notna()]
combined_df

,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency,original,mutant
0,1,169476354,T>C,snv,1000230073,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,C
1,1,169476354,T>C,snv,1000230073,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,T
2,1,169475878,T>C,snv,1000304891,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,T
3,1,169475878,T>C,snv,1000304891,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,C
4,1,169483460,T>C,snv,1000361639,normal,by-frequency;by-alfa,intron_variant,SLC19A2,C:0.:0:ALFA,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...
22128,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,A
22129,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,C
22130,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,T
22131,1,169481070,C>A,snv,999949244,normal,by-frequency;by-alfa,intron_variant,SLC19A2,A:0.000004:1:TOPMED|A:0.:0:ALFA,C,C


### Generating DNA Sequence for each example

In [18]:

print("\n--- Step 3: Generating DNA Sequences for Each Row ---")

# --- Ensure prerequisites exist ---
if 'combined_df' not in locals() or combined_df is None:
     print("ERROR: DataFrame 'combined_df' is not available.")
# (Add checks for wt_genomic_sequence_region, genomic_region_start if needed)
# ...
elif 'wt_genomic_sequence_region' not in locals() or wt_genomic_sequence_region is None:
     print("ERROR: WT Genomic sequence ('wt_genomic_sequence_region') is not available.")
elif 'genomic_region_start' not in locals() or genomic_region_start is None:
     print("ERROR: Genomic region start position ('genomic_region_start') is not defined.")
else:
    # --- Initialize ---
    generated_sequences = []
    generation_errors = 0
    wt_len = len(wt_genomic_sequence_region) # Cache length

    # --- Iterate through DataFrame ---
    # Using itertuples provides row index via row.Index
    for row in combined_df.itertuples(index=True):
        generated_seq_for_row = None # Default to None
        row_index = row.Index        # Get the original DataFrame index for error reporting
        snp_id_for_error = 'N/A'     # Default SNP ID if reading fails

        try:
            # --- Read identifiers and critical data FIRST ---
            # Attempt to read SNP ID early for better error messages
            snp_id_for_error = str(getattr(row, 'snp_id', f'Index_{row_index}')).strip()

            # --- Robustly handle 'pos' column ---
            pos_val = getattr(row, 'pos', None) # Get raw value
            if pd.isna(pos_val):
                # Handle NaN values for position
                print(f"  ERROR (snp {snp_id_for_error}, Index {row_index}): 'pos' column is NaN. Skipping row.")
                generation_errors += 1
                generated_sequences.append(None) # Append None as sequence cannot be generated
                continue # Skip to the next row
            try:
                # Attempt integer conversion only if not NaN
                pos = int(pos_val)
            except (ValueError, TypeError):
                # Handle values that are not NaN but cannot be converted to int
                print(f"  ERROR (snp {snp_id_for_error}, Index {row_index}): 'pos' value '{pos_val}' is not a valid integer. Skipping row.")
                generation_errors += 1
                generated_sequences.append(None) # Append None
                continue # Skip to the next row
            # --- End of robust 'pos' handling ---

            # Read other necessary fields
            original_allele = str(getattr(row, 'original', '')).upper()
            mutant_allele = str(getattr(row, 'mutant', '')).upper()
            significance = str(getattr(row, 'clinical_significance', '')).lower()

            # --- Check Significance ---
            if significance == 'normal':
                generated_seq_for_row = wt_genomic_sequence_region
            else:
                # --- Process non-normal variants ---
                index_0based = pos - genomic_region_start

                # --- Handle Insertion (original == '-') ---
                if original_allele == '-':
                    if not (0 <= index_0based <= wt_len):
                         print(f"  ERROR (snp {snp_id_for_error}, pos {pos}, Index {row_index}): Insertion index {index_0based} out of bounds [0, {wt_len}].")
                         generation_errors += 1
                    else:
                         generated_seq_for_row = wt_genomic_sequence_region[:index_0based] + \
                                                 mutant_allele + \
                                                 wt_genomic_sequence_region[index_0based:]

                # --- Handle Substitution/Deletion (original != '-' and not empty) ---
                elif original_allele:
                    ref_len = len(original_allele)
                    if not (0 <= index_0based < wt_len and index_0based + ref_len <= wt_len):
                        print(f"  ERROR (snp {snp_id_for_error}, pos {pos}, Index {row_index}): Original allele '{original_allele}' segment out of bounds for WT sequence.")
                        generation_errors += 1
                    else:
                        ref_from_genome = wt_genomic_sequence_region[index_0based : index_0based + ref_len].upper()
                        if ref_from_genome == original_allele:
                            # Match: Apply mutation
                            if mutant_allele == '-': # Deletion
                                generated_seq_for_row = wt_genomic_sequence_region[:index_0based] + \
                                                        wt_genomic_sequence_region[index_0based + ref_len:]
                            else: # Substitution
                                generated_seq_for_row = wt_genomic_sequence_region[:index_0based] + \
                                                        mutant_allele + \
                                                        wt_genomic_sequence_region[index_0based + ref_len:]
                        else:
                            # Mismatch: This is the validation error
                            print(f"  ERROR (snp {snp_id_for_error}, pos {pos}, Index {row_index}): Validation failed. Expected WT '{original_allele}' but found '{ref_from_genome}'.")
                            generation_errors += 1
                            # generated_seq_for_row remains None
                else:
                    # Handle case where original is empty but not '-'
                    print(f"  WARN (snp {snp_id_for_error}, pos {pos}, Index {row_index}): Original allele is empty but not '-'. Cannot process.")
                    generation_errors += 1
                    # generated_seq_for_row remains None

        except Exception as e:
            # Catch any other unexpected errors (should be less frequent now)
            print(f"  CRITICAL ERROR processing row (SNP/ID: {snp_id_for_error}, Index: {row_index}): {e}")
            generation_errors += 1
            generated_seq_for_row = None # Ensure None on critical error

        # Append result for this row
        generated_sequences.append(generated_seq_for_row)

    # --- Add the new column to the DataFrame ---
    if len(generated_sequences) == len(combined_df):
        combined_df['generated_dna'] = generated_sequences
        print(f"\n'generated_dna' column added successfully.")
    else:
        print(f"\nERROR: Length mismatch! Generated sequences ({len(generated_sequences)}) != DataFrame rows ({len(combined_df)}). Column not added.")
        generation_errors = len(combined_df) # Indicate major failure

    # --- Report Errors ---
    print(f"\n--- Sequence Generation Finished ---")
    print(f"Total errors encountered during sequence generation: {generation_errors}")

    # --- Display Head ---
    if 'generated_dna' in combined_df.columns:
        print("\nDataFrame head including 'generated_dna' column info:")
        display_df = combined_df[['#chr', 'pos', 'snp_id', 'original', 'mutant', 'clinical_significance']].copy()
        display_df['generated_dna_type'] = combined_df['generated_dna'].apply(lambda x: type(x).__name__)
        display_df['generated_dna_len'] = combined_df['generated_dna'].apply(lambda x: len(x) if isinstance(x, str) else 0)
        print(display_df.head(10))
        failed_rows = combined_df['generated_dna'].isna().sum()
        if failed_rows > 0:
             print(f"\nNote: {failed_rows} row(s) have 'None'/'NaN' in 'generated_dna' due to errors.")
    else:
        print("\n'generated_dna' column was not added due to errors.")


--- Step 3: Generating DNA Sequences for Each Row ---

'generated_dna' column added successfully.

--- Sequence Generation Finished ---
Total errors encountered during sequence generation: 0

DataFrame head including 'generated_dna' column info:
  #chr        pos      snp_id original mutant clinical_significance  \
0    1  169476354  1000230073        T      C                   NaN   
1    1  169476354  1000230073        T      T                normal   
2    1  169475878  1000304891        T      T                normal   
3    1  169475878  1000304891        T      C                   NaN   
4    1  169483460  1000361639        T      T                normal   
5    1  169483460  1000361639        T      C                   NaN   
6    1  169475969  1000582129        G      G                normal   
7    1  169475969  1000582129        G      C                   NaN   
8    1  169468609  1000671246        T      A                   NaN   
9    1  169468609  1000671246        T     

In [21]:
# Get the size before filtering
original_size = len(combined_df)
print(f"Original DataFrame size: {original_size} rows")

# Remove rows where generated_dna is NaN or empty
combined_df = combined_df.dropna(subset=['generated_dna'])
# Also remove rows where generated_dna is an empty string
combined_df = combined_df[combined_df['generated_dna'] != '']

# Get the size after filtering
filtered_size = len(combined_df)
print(f"Filtered DataFrame size: {filtered_size} rows")
print(f"Removed {original_size - filtered_size} rows")

combined_df.head()

Original DataFrame size: 22133 rows
Filtered DataFrame size: 22133 rows
Removed 0 rows


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency,original,mutant,generated_dna
0,1,169476354,T>C,snv,1000230073,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,C,GGCAAACCTAACATTTTGAACCCCCTTCTGTAGTATTACTTACTGA...
1,1,169476354,T>C,snv,1000230073,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,T,GGCAAACCTAACATTTTGAACCCCCTTCTGTAGTATTACTTACTGA...
2,1,169475878,T>C,snv,1000304891,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,T,GGCAAACCTAACATTTTGAACCCCCTTCTGTAGTATTACTTACTGA...
3,1,169475878,T>C,snv,1000304891,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,C,GGCAAACCTAACATTTTGAACCCCCTTCTGTAGTATTACTTACTGA...
4,1,169483460,T>C,snv,1000361639,normal,by-frequency;by-alfa,intron_variant,SLC19A2,C:0.:0:ALFA,T,T,GGCAAACCTAACATTTTGAACCCCCTTCTGTAGTATTACTTACTGA...



### Mapping the DNA TO RNA

In [29]:
# getting the exons coordinates

import requests
import json
import sys # For printing error messages to stderr

# --- Target Transcript ---
# USE THE ENSEMBL ID CORRESPONDING TO NM_025243.4
# transcript_id = "ENST00000644224" this for SLC19A3
# NM_001319667.1 for slc19a2
transcript_id = "ENST00000236137" # for SLC19A2

# --- Ensembl REST API Configuration ---
server = "https://rest.ensembl.org"
# We use the lookup/id endpoint with the 'expand=exons' parameter
ext = f"/lookup/id/{transcript_id}?expand=exons"
headers = {"Content-Type": "application/json"}

print(f"Querying Ensembl REST API for exon genomic coordinates for: {transcript_id}")
print(f"URL: {server + ext}")

genomic_exons = [] # List to store results: [(chromosome, start, end, strand), ...]

try:
    # Make the GET request to the Ensembl API
    r = requests.get(server + ext, headers=headers, timeout=30) # Increased timeout

    # Check if the request was successful (status code 200-299)
    r.raise_for_status()

    # Parse the JSON response
    decoded = r.json()

    # --- Extract Exon Information ---
    exons_data = decoded.get('Exon')

    if not exons_data:
        print("Error: No 'Exon' data found in the Ensembl response.", file=sys.stderr)
        print("Full response:", json.dumps(decoded, indent=2))
    else:
        print(f"\nFound {len(exons_data)} exons in the response.")
        transcript_strand = decoded.get('strand')

        for i, exon in enumerate(exons_data):
            chromosome = exon.get('seq_region_name')
            start = exon.get('start')
            end = exon.get('end')
            strand = exon.get('strand')
            exon_id = exon.get('id') # Ensembl exon ID

            if all([chromosome, start, end, strand is not None]):
                 try:
                     start = int(start)
                     end = int(end)
                     strand = int(strand)
                     genomic_exons.append((chromosome, start, end, strand))
                     # Use Ensembl exon ID in printout
                     print(f"  Exon {i+1} ({exon_id}): Chr {chromosome}:{start}-{end} Strand:{strand}")
                 except ValueError:
                      print(f"  Warning: Could not convert coordinates to integers for exon {exon_id}", file=sys.stderr)
            else:
                print(f"  Warning: Missing data for exon {i+1} ({exon_id}). Skipping.", file=sys.stderr)
                print(f"    Data found: chr={chromosome}, start={start}, end={end}, strand={strand}")

        # --- Sort Exons Biologically ---
        if genomic_exons:
             genomic_exons.sort(key=lambda x: x[1]) # Sort by start coordinate
             print("\nExons sorted by genomic start coordinate:")
             for chrom, start_coord, end_coord, strand_coord in genomic_exons:
                  print(f"  Chr {chrom}:{start_coord}-{end_coord} (Strand {strand_coord})")
        else:
             print("\nNo valid exons extracted to sort.")


except requests.exceptions.RequestException as e:
    print(f"\nHTTP Request failed: {e}", file=sys.stderr)
    if hasattr(e, 'response') and e.response is not None:
         print(f"Response status code: {e.response.status_code}", file=sys.stderr)
         print(f"Response content: {e.response.text}", file=sys.stderr)
except json.JSONDecodeError:
    print("Failed to decode JSON response.", file=sys.stderr)
    print(f"Response text was: {r.text}", file=sys.stderr)
except KeyError as e:
    print(f"Missing expected key in JSON response: {e}", file=sys.stderr)
    print("Full response structure:", json.dumps(decoded, indent=2))
except Exception as e:
    print(f"An unexpected error occurred: {e}", file=sys.stderr)


# --- Final Result ---
if genomic_exons:
    print(f"\nSuccessfully retrieved {len(genomic_exons)} genomic exon coordinates.")
    # Store this list for the next step (mapping function)
    # Example: You might assign this to a variable accessible by the mapping function
    # global_genomic_exons_map[transcript_id] = genomic_exons
else:
    print("\nFailed to retrieve genomic exon coordinates.")

Querying Ensembl REST API for exon genomic coordinates for: ENST00000236137
URL: https://rest.ensembl.org/lookup/id/ENST00000236137?expand=exons

Found 6 exons in the response.
  Exon 1 (ENSE00003821119): Chr 1:169485563-169485944 Strand:-1
  Exon 2 (ENSE00000789596): Chr 1:169477155-169477757 Strand:-1
  Exon 3 (ENSE00000789595): Chr 1:169469964-169470186 Strand:-1
  Exon 4 (ENSE00000789594): Chr 1:169468644-169468836 Strand:-1
  Exon 5 (ENSE00001021778): Chr 1:169468111-169468252 Strand:-1
  Exon 6 (ENSE00001357740): Chr 1:169463909-169465977 Strand:-1

Exons sorted by genomic start coordinate:
  Chr 1:169463909-169465977 (Strand -1)
  Chr 1:169468111-169468252 (Strand -1)
  Chr 1:169468644-169468836 (Strand -1)
  Chr 1:169469964-169470186 (Strand -1)
  Chr 1:169477155-169477757 (Strand -1)
  Chr 1:169485563-169485944 (Strand -1)

Successfully retrieved 6 genomic exon coordinates.


In [61]:

genomic_exons = [
    ('1', 169463909, 169465977, -1),
    ('1',169468111, 169468252, -1),
    ('1', 169468644, 169468836, -1),
    ('1', 169469964, 169470186, -1),
    ('1', 169477155, 169477757, -1),
    ('1', 169485563, 169485944, -1),
    
]


# Determine genomic range needed
chromosome_acc = "NC_000001.11" # Chromosome 1 accession
min_coord = min(e[1] for e in genomic_exons)
max_coord = max(e[2] for e in genomic_exons)
print(f"Need genomic region: {chromosome_acc} from {min_coord} to {max_coord}")

# --- 1. Fetch Genomic DNA Sequence (+ Strand) ---
print("\nFetching positive strand genomic DNA sequence...")
genomic_seq_record = None
try:
    time.sleep(entrez_delay)
    handle = Entrez.efetch(db="nucleotide",
                           id=chromosome_acc,
                           rettype="fasta",
                           retmode="text",
                           seq_start=min_coord,
                           seq_stop=max_coord,
                           strand=1) # strand=1 fetches the positive strand sequence
    genomic_seq_record = SeqIO.read(io.StringIO(handle.read()), "fasta")
    handle.close()
    print(f"Successfully fetched {len(genomic_seq_record.seq)} bp of genomic sequence.")
    print(f"Genomic (+) strand (first 60 bp): {genomic_seq_record.seq[:60]}...")
    print(f"Genomic (+) strand (last 60 bp): {genomic_seq_record.seq[-60:]}...")

except Exception as e:
    print(f"Error fetching genomic sequence: {e}")
    # If fetching fails, prompt user (as per original request)
    print("\nCould not automatically fetch genomic sequence.")
    print("Please provide the sequence if you have it, or we cannot proceed with this test.")
    # In a real script, you might load from a file here if fetch fails
    genomic_seq_record = None # Ensure it's None if fetch failed



Need genomic region: NC_000001.11 from 169463909 to 169485944

Fetching positive strand genomic DNA sequence...
Successfully fetched 22036 bp of genomic sequence.
Genomic (+) strand (first 60 bp): TTTTGATTAAAAAGAGAAAATATACTGTAAAATATTTATTTAATAAAAATAATTTTATAA...
Genomic (+) strand (last 60 bp): TTCTGGACTCGCCGCCGCCTCCGGCTACAGAACCCCCAGCTTTACCCTACAGACGCCTCT...


In [35]:

# --- Check prerequisites ---
if 'wt_genomic_sequence_region' not in locals() or not isinstance(wt_genomic_sequence_region, str):
     print("ERROR: WT Genomic sequence ('wt_genomic_sequence_region') is not available or not a string.")
elif 'genomic_region_start' not in locals() or not isinstance(genomic_region_start, int):
     print("ERROR: Genomic region start ('genomic_region_start') is not defined or not an integer.")
else:
    # --- Initialize storage ---
    exon_data = {
        'Exon_Index': [],
        'Genomic_Start': [],
        'Genomic_End': [],
        'Genomic_Strand': [],
        'Relative_Start_Index': [], # 0-based start in wt_genomic_sequence_region
        'Relative_End_Index': [],   # 0-based end (exclusive) in wt_genomic_sequence_region
        'Sequence': []
    }
    extraction_errors = 0
    wt_len = len(wt_genomic_sequence_region)

    print(f"WT Region Start Coordinate: {genomic_region_start}")
    print(f"WT Region Length: {wt_len}")

    # --- Iterate through exons ---
    for i, (chrom, exon_start, exon_end, strand) in enumerate(genomic_exons):
        exon_index = i + 1 # 1-based index for exons
        print(f"\nProcessing Exon {exon_index}: Genomic {exon_start}-{exon_end}")

        # Calculate 0-based relative indices for slicing
        relative_start_index = exon_start - genomic_region_start
        # The end index for Python slicing needs to be one past the last base
        relative_end_index = exon_end - genomic_region_start + 1

        print(f"  Calculated Relative Indices: [{relative_start_index}:{relative_end_index}]")

        # --- Bounds Check ---
        valid_indices = True
        if not (0 <= relative_start_index < wt_len):
            print(f"  ERROR: Relative Start Index ({relative_start_index}) is out of bounds [0, {wt_len-1}).")
            valid_indices = False
        # End index can be equal to wt_len for slicing up to the end
        if not (0 < relative_end_index <= wt_len):
             print(f"  ERROR: Relative End Index ({relative_end_index}) is out of bounds (0, {wt_len}].")
             valid_indices = False
        if valid_indices and relative_start_index >= relative_end_index:
            print(f"  ERROR: Relative Start Index ({relative_start_index}) is not less than Relative End Index ({relative_end_index}).")
            valid_indices = False

        # --- Extract Sequence if Valid ---
        extracted_sequence = None
        if valid_indices:
            try:
                extracted_sequence = wt_genomic_sequence_region[relative_start_index:relative_end_index]
                print(f"  Successfully extracted sequence (length: {len(extracted_sequence)}).")
            except Exception as e:
                 print(f"  ERROR during slicing: {e}")
                 valid_indices = False # Mark as invalid if slicing fails unexpectedly

        if not valid_indices:
            extraction_errors += 1
            # Ensure None is appended if extraction failed
            relative_start_index = None
            relative_end_index = None
            extracted_sequence = None
            print(f"  Failed to extract sequence for Exon {exon_index}.")


        # --- Store Data ---
        exon_data['Exon_Index'].append(exon_index)
        exon_data['Genomic_Start'].append(exon_start)
        exon_data['Genomic_End'].append(exon_end)
        exon_data['Genomic_Strand'].append(strand)
        exon_data['Relative_Start_Index'].append(relative_start_index)
        exon_data['Relative_End_Index'].append(relative_end_index)
        exon_data['Sequence'].append(extracted_sequence)

    # --- Create DataFrame ---
    exons_df = pd.DataFrame(exon_data)

    print("\n--- Exon Extraction Finished ---")
    if extraction_errors > 0:
        print(f"WARNING: {extraction_errors} error(s) occurred during exon extraction.")

    # --- Display Results ---
    print("\nDataFrame of Extracted Exons:")
    # Display relevant columns, Sequence length instead of full sequence
    display_df = exons_df.drop(columns=['Sequence'], errors='ignore').copy()
    if 'Sequence' in exons_df.columns:
         display_df['Sequence_Length'] = exons_df['Sequence'].apply(lambda x: len(x) if isinstance(x, str) else 0)
    print(display_df)

    # Optional: Check if any sequences are None
    failed_extractions = exons_df['Sequence'].isna().sum()
    if failed_extractions > 0:
        print(f"\nNote: {failed_extractions} exon sequence(s) could not be extracted due to errors (marked as None/NaN).")

WT Region Start Coordinate: 169461909
WT Region Length: 26062

Processing Exon 1: Genomic 169463909-169465977
  Calculated Relative Indices: [2000:4069]
  Successfully extracted sequence (length: 2069).

Processing Exon 2: Genomic 169468111-169468252
  Calculated Relative Indices: [6202:6344]
  Successfully extracted sequence (length: 142).

Processing Exon 3: Genomic 169468644-169468836
  Calculated Relative Indices: [6735:6928]
  Successfully extracted sequence (length: 193).

Processing Exon 4: Genomic 169469964-169470186
  Calculated Relative Indices: [8055:8278]
  Successfully extracted sequence (length: 223).

Processing Exon 5: Genomic 169477155-169477757
  Calculated Relative Indices: [15246:15849]
  Successfully extracted sequence (length: 603).

Processing Exon 6: Genomic 169485563-169485944
  Calculated Relative Indices: [23654:24036]
  Successfully extracted sequence (length: 382).

--- Exon Extraction Finished ---

DataFrame of Extracted Exons:
   Exon_Index  Genomic_Start

In [41]:
# Converting the dna sequence to mrna sequence using the exons coordinates

# Proceed only if genomic sequence was obtained
if genomic_seq_record: 

    # (Code for complementary strand can stay as is)
    # ...

    # --- 3. Simulate Transcription (Negative Strand Gene) - REVISED SLICING ---
    print("\nSimulating transcription from positive strand using negative strand logic (REVISED SLICING)...")
    simulated_mrna_parts = []
    min_coord = min(e[1] for e in genomic_exons) # Ensure min_coord is defined here too

    # Iterate through exons in REVERSE order (highest genomic coord first) for negative strand
    for chrom, start, end, strand in reversed(genomic_exons):
        if strand == -1:
            print(f"  Processing exon {start}-{end} (Strand {strand})") # Debug print

            # Calculate 0-based slice indices relative to the START of our fetched sequence
            # Start index (inclusive):
            slice_start_0based = start - min_coord
            # End index (inclusive):
            slice_end_0based_inclusive = end - min_coord
            # Correct Python slice end index (exclusive):
            slice_end_exclusive = slice_end_0based_inclusive + 1

            print(f"    Relative slice indices (0-based): [{slice_start_0based}:{slice_end_exclusive}]") # Debug print

            # Ensure indices are within bounds of the fetched sequence
            if slice_start_0based < 0 or slice_end_exclusive > len(genomic_seq_record.seq):
                 print(f"    ERROR: Calculated slice [{slice_start_0based}:{slice_end_exclusive}] is out of bounds for fetched sequence length {len(genomic_seq_record.seq)}")
                 continue # Skip this exon if indices are wrong

            # Extract the segment from the POSITIVE strand genomic sequence
            exon_seq_on_pos_strand = genomic_seq_record.seq[slice_start_0based : slice_end_exclusive]
            print(f"    Extracted (+) strand length: {len(exon_seq_on_pos_strand)}") # Debug print
            # print(f"    Extracted (+) strand seq: {exon_seq_on_pos_strand[:10]}...") # Optional further debug

            # Get the REVERSE COMPLEMENT
            exon_seq_for_mrna = exon_seq_on_pos_strand.reverse_complement()
            # print(f"    Reverse Complement seq: {exon_seq_for_mrna[:10]}...") # Optional further debug

            # Append the resulting sequence string
            simulated_mrna_parts.append(str(exon_seq_for_mrna))
        else:
            print(f"Warning: Found exon with unexpected strand: {strand}")


    # Join the parts to form the full mRNA sequence
    simulated_mrna_seq_str = "".join(simulated_mrna_parts)
    simulated_mrna_seq_obj = Seq(simulated_mrna_seq_str)

    print(f"\nSimulated mRNA sequence length: {len(simulated_mrna_seq_obj)} bp")
    print(f"Simulated mRNA (first 60 bp): {simulated_mrna_seq_obj[:60]}...")
    print(f"Simulated mRNA (last 60 bp): ...{simulated_mrna_seq_obj[-60:]}")

    # --- 4. Verification (Optional - Compare with actual NM_025243.4) ---
    # (Keep the verification code the same, but add the comparison script provided by user)

    print("\n--- Verification Comparison ---")
    try:
        print("Fetching actual NM_006996.3 sequence for comparison...")
        time.sleep(entrez_delay)
        handle_mrna = Entrez.efetch(db="nucleotide", id="NM_006996.3", rettype="fasta", retmode="text")
        actual_mrna_record = SeqIO.read(io.StringIO(handle_mrna.read()), "fasta")
        handle_mrna.close()

        # --- Comparison Logic ---
        actual_seq = str(actual_mrna_record.seq) # Use string for direct comparison
        actual_len = len(actual_seq)
        simulated_len = len(simulated_mrna_seq_str)
        min_len = min(actual_len, simulated_len)
        max_len = max(actual_len, simulated_len)

        first_diff_pos = None
        diff = 0
        for i in range(min_len):
            if actual_seq[i] != simulated_mrna_seq_str[i]:
                if first_diff_pos is None:
                    first_diff_pos = i
                diff += 1

        length_diff = max_len - min_len
        total_diff = diff + length_diff

        print(f"Sequence length comparison: Actual={actual_len}, Simulated={simulated_len}")

        if total_diff == 0:
             print("SUCCESS: Simulated mRNA sequence MATCHES the actual NM_006996.3 sequence!")
        else:
             print("MISMATCH: Simulated mRNA sequence does NOT match the actual NM_006996.3sequence.")
             print(f"  First difference occurs at position: {first_diff_pos if first_diff_pos is not None else 'N/A (Length Mismatch Only)'}")
             print(f"  Differences in overlapping portion ({min_len} bp): {diff}")
             print(f"  Length difference: {length_diff}")
             print(f"  Total differences: {total_diff}")
             if max_len > 0:
                  print(f"  Percent match: {100 * (max_len - total_diff) / max_len:.2f}%")
             else:
                  print("  Percent match: N/A (Zero length)")

    except Exception as e:
        print(f"Could not fetch or compare actual mRNA sequence: {e}")


Simulating transcription from positive strand using negative strand logic (REVISED SLICING)...
  Processing exon 169485563-169485944 (Strand -1)
    Relative slice indices (0-based): [21654:22036]
    Extracted (+) strand length: 382
  Processing exon 169477155-169477757 (Strand -1)
    Relative slice indices (0-based): [13246:13849]
    Extracted (+) strand length: 603
  Processing exon 169469964-169470186 (Strand -1)
    Relative slice indices (0-based): [6055:6278]
    Extracted (+) strand length: 223
  Processing exon 169468644-169468836 (Strand -1)
    Relative slice indices (0-based): [4735:4928]
    Extracted (+) strand length: 193
  Processing exon 169468111-169468252 (Strand -1)
    Relative slice indices (0-based): [4202:4344]
    Extracted (+) strand length: 142
  Processing exon 169463909-169465977 (Strand -1)
    Relative slice indices (0-based): [0:2069]
    Extracted (+) strand length: 2069

Simulated mRNA sequence length: 3612 bp
Simulated mRNA (first 60 bp): AGAGGCGTC

In [42]:
# Store the exons dataframe with a timestamp in the filename
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
exons_df.to_csv(f"exons_data_{timestamp}.csv", index=False)
print(f"Exons data saved to exons_data_{timestamp}.csv")

# Display the dataframe
exons_df

# Note these exons are still on the positive strand (if you need to reverse them, you can do so by using the reverse_complement method) 

Exons data saved to exons_data_20250505_171511.csv


,Exon_Index,Genomic_Start,Genomic_End,Genomic_Strand,Relative_Start_Index,Relative_End_Index,Sequence
0,1,169463909,169465977,-1,2000,4069,TTTTGATTAAAAAGAGAAAATATACTGTAAAATATTTATTTAATAA...
1,2,169468111,169468252,-1,6202,6344,CTGAGTGGTAATTTCTAATCCAAGGCCACTGGCATCTACCACAATT...
2,3,169468644,169468836,-1,6735,6928,GTTGCTATCGTGATGAGTAACATGTAGATGATTCTGAAGACAACAT...
3,4,169469964,169470186,-1,8055,8278,CCAGTAAGGTTGAAACGGCCTCCACGCCACCATTATAGATAGCAGC...
4,5,169477155,169477757,-1,15246,15849,CGGTTCCTCCACGGGAGGCTCCTCCATATTTAGAGGGATTTTTGAC...
5,6,169485563,169485944,-1,23654,24036,CTCCCTCTCGGTCAGGTTCTTGTCCGGCCCCAGCAGGTACGGGGTC...


### Generating Full mRNAs

In [43]:

# Convert the DNA reverse complement to RNA (replace T with U) and store as mRNA_WT

from Bio.Seq import Seq # Import Seq object for reverse_complement

rna_sequence_youssef = ''.join(exons_df['Sequence'])
reverse_complement_exons = str(Seq(rna_sequence_youssef).reverse_complement())
reverse_complement_exons
mRNA_WT = reverse_complement_exons.replace('T', 'U')
print("mRNA wild type sequence length:", len(mRNA_WT))
print("First 50 nucleotides of mRNA:", mRNA_WT[:50])
print("Last 50 nucleotides of mRNA:", mRNA_WT[-50:])


mRNA wild type sequence length: 3612
First 50 nucleotides of mRNA: AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGGCGGC
Last 50 nucleotides of mRNA: AUUUUUAUUAAAUAAAUAUUUUACAGUAUAUUUUCUCUUUUUAAUCAAAA


In [44]:
mRNA_Ready = combined_df.copy()
# Create a new column for generated mRNA sequences
mRNA_Ready['generated_mRNA'] = None

def calculate_relative_position(pos, exon_start):
    return int(pos) - int(exon_start)
     
# Function to check if a position is within any exon
def is_position_in_exon(pos):
    '''
    Check if a position is within any exon and return the exon number, genomic start, genomic end, sequence, and relative position.
    
    Args:
        pos (int): Genomic position to check
        
    Returns:
        tuple: (True/False, exon_index (1-based), genomic_start, genomic_end, sequence, relative_position)
    '''
    for index, row in exons_df.iterrows():
        if row['Genomic_Start'] <= pos <= row['Genomic_End']:
            relative_position = calculate_relative_position(pos, row['Genomic_Start'])
            return True, index+1, row['Genomic_Start'], row['Genomic_End'], row['Sequence'], relative_position
    return False, None, None, None, None, None


def fuse_exons(exon_index_1_based, sequence):
    # fuse the exons to create the full sequence
    exons_fused = ''
    for index,row in exons_df.iterrows():
        if index+1 != exon_index_1_based:
            exons_fused += row['Sequence']
        else:
            exons_fused += sequence
    return exons_fused

# Initialize counters outside the function
substitutions = 0
delted_stuff = 0
inserted_in_exon = 0

def mutate_exon(exon_sequence, relative_position, original, mutant):
    global substitutions, delted_stuff, inserted_in_exon
    mutated_exon = list(exon_sequence)
    
    if mutant != '-' and original != '-':
        # Handle substitutions of different lengths
        # First remove the original sequence
        del mutated_exon[relative_position:relative_position + len(original)]
        # Then insert the new sequence
        for i, char in enumerate(mutant):
            mutated_exon.insert(relative_position + i, char)
        substitutions += 1

    elif mutant == '-':  # Deletion
        # Remove the characters at the position
        del mutated_exon[relative_position:relative_position + len(original)]
        delted_stuff += 1
    
    elif original == '-':  # Insertion
        # Just insert the new sequence at the position
        for i, char in enumerate(mutant):
            mutated_exon.insert(relative_position + i, char)
        inserted_in_exon += 1
    
    return ''.join(mutated_exon)

def mutate_reverse_complemnet_and_return_mRNA(exon_index_1_based, exon_sequence, relative_position, original, mutant):
    # this takes the exon sequence on positive strand, mutate it and joins with other exons, then reverse complements it converts it to mRNA
    mutated_exon = mutate_exon(exon_sequence, relative_position, original, mutant)
    mutated_axons_on_template_strand = fuse_exons(exon_index_1_based, mutated_exon)
    tranlsated_sequence = str(Seq(mutated_axons_on_template_strand).reverse_complement())
    mRNA = tranlsated_sequence.replace('T', 'U')
    return mRNA

# Iterate through each row in the dataframe and generate the mRNA sequence for each row (complement based)
Non_matches = 0
skipped_cause_of_normal = 0
skipped_cause_of_not_in_exon = 0
matched_in_exon_and_mutated = 0 

for idx, row in mRNA_Ready.iterrows():
    if row['clinical_significance'] == 'normal':
        # For normal variants, use the wild type mRNA
        mRNA_Ready.at[idx, 'generated_mRNA'] = mRNA_WT
        skipped_cause_of_normal += 1
    else:
        # For non-normal variants, check if position is within any exon
        pos = int(row['pos'])
        is_in_exon, exon_index_1_based, genomic_start, genomic_end, exon_sequence, relative_position = is_position_in_exon(pos)
        if not is_in_exon:
            mRNA_Ready.at[idx, 'generated_mRNA'] = mRNA_WT
            skipped_cause_of_not_in_exon += 1
        else:
            len_of_original = len(row['original'])
            if row['original'] == '-':
                mRNA = mutate_reverse_complemnet_and_return_mRNA(exon_index_1_based, exon_sequence, relative_position, row['original'], row['mutant'])
                mRNA_Ready.at[idx, 'generated_mRNA'] = mRNA
            
            elif exon_sequence[relative_position:relative_position+len_of_original] == row['original']:
                print(f'''Mutation of SNP_ID: {row["snp_id"]} exists in exon {exon_index_1_based}
                      And it Matches, we are going to change {row['original']} to {row['mutant']}''')
                
                mRNA = mutate_reverse_complemnet_and_return_mRNA(exon_index_1_based, exon_sequence, relative_position, row['original'], row['mutant'])
                mRNA_Ready.at[idx, 'generated_mRNA'] = mRNA
                matched_in_exon_and_mutated += 1
            else:
                print(f'''Mutation of SNP_ID: {row["snp_id"]} exists in exon {exon_index_1_based}
                      And it does not match, expected {row['original']} and found {exon_sequence[relative_position:relative_position+len_of_original]}''')
                Non_matches += 1
                mRNA_Ready.at[idx, 'generated_mRNA'] = mRNA_WT  # Use wild type for non-matches
                
total = matched_in_exon_and_mutated + skipped_cause_of_normal + skipped_cause_of_not_in_exon + Non_matches
# Print summary of mRNA generation
print(f'''generated mRNA on negative strand, matched in exon and mutated: {matched_in_exon_and_mutated}, 
      got normal mRNA: {skipped_cause_of_normal},
      skipped because not in exon: {skipped_cause_of_not_in_exon},
      non matches: {Non_matches},
      substitutions: {substitutions},
      inserted in exon: {inserted_in_exon},
      deleted in exon: {delted_stuff},
      total: {total}
''')


Mutation of SNP_ID: 1001321069 exists in exon 1
                      And it Matches, we are going to change G to C
Mutation of SNP_ID: 1001624959 exists in exon 1
                      And it Matches, we are going to change C to A
Mutation of SNP_ID: 1001853470 exists in exon 5
                      And it Matches, we are going to change T to C
Mutation of SNP_ID: 1003505341 exists in exon 1
                      And it Matches, we are going to change G to A
Mutation of SNP_ID: 1003505341 exists in exon 1
                      And it Matches, we are going to change G to T
Mutation of SNP_ID: 1004026015 exists in exon 1
                      And it Matches, we are going to change G to A
Mutation of SNP_ID: 1004026015 exists in exon 1
                      And it Matches, we are going to change G to C
Mutation of SNP_ID: 1006414225 exists in exon 6
                      And it Matches, we are going to change C to T
Mutation of SNP_ID: 1007470423 exists in exon 1
                      An

In [45]:
# Save to pickle file
pickle_filename = f"csv_files/mRNA_Ready_{timestamp}.pkl"
mRNA_Ready.to_pickle(pickle_filename)
print(f"Data saved to pickle file: {pickle_filename}")

# Save to CSV file
csv_filename = f"csv_files/mRNA_Ready_{timestamp}.csv"
mRNA_Ready.to_csv(csv_filename, index=False)
print(f"Data saved to CSV file: {csv_filename}")


Data saved to pickle file: csv_files/mRNA_Ready_20250505_171511.pkl
Data saved to CSV file: csv_files/mRNA_Ready_20250505_171511.csv


### Reverse Complement The DNA

In [46]:

print("Reverse complementing DNA sequences...")

# Function to reverse complement a DNA sequence
def reverse_complement(dna_sequence):
    if isinstance(dna_sequence, str):
        return str(Seq(dna_sequence).reverse_complement())
    return None


# Using .copy() to avoid SettingWithCopyWarning
mRNA_Ready_copy = mRNA_Ready.copy()
mRNA_Ready_copy['generated_dna'] = mRNA_Ready_copy['generated_dna'].apply(reverse_complement)

# Replace the original DataFrame with the modified copy
mRNA_Ready = mRNA_Ready_copy

print(f"Completed reverse complementing {len(mRNA_Ready)} DNA sequences")

# Display a sample of the first few rows to verify the transformation
print("\nSample of reverse complemented DNA sequences:")
sample_df = mRNA_Ready.head(3)[['snp_id', 'generated_dna']]
for _, row in sample_df.iterrows():
    print(f"SNP ID: {row['snp_id']}")
    print(f"DNA (first 50 chars): {row['generated_dna'][:50]}...")
    print()


Reverse complementing DNA sequences...
Completed reverse complementing 22133 DNA sequences

Sample of reverse complemented DNA sequences:
SNP ID: 1000230073
DNA (first 50 chars): ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTCCGCT...

SNP ID: 1000230073
DNA (first 50 chars): ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTCCGCT...

SNP ID: 1000304891
DNA (first 50 chars): ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTCCGCT...



In [47]:
mRNA_Ready

,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency,original,mutant,generated_dna,generated_mRNA
0,1,169476354,T>C,snv,1000230073,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,C,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
1,1,169476354,T>C,snv,1000230073,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000013:2:GnomAD_genomes|C:0.000008:2:TOPME...,T,T,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
2,1,169475878,T>C,snv,1000304891,normal,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,T,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
3,1,169475878,T>C,snv,1000304891,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.000007:1:GnomAD_genomes|C:0.000008:2:TOPME...,T,C,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
4,1,169483460,T>C,snv,1000361639,normal,by-frequency;by-alfa,intron_variant,SLC19A2,C:0.:0:ALFA,T,T,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22128,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,A,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
22129,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,C,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
22130,1,169485475,"G>A,C,T",snv,999809738,NaN,by-frequency;by-alfa;by-cluster,intron_variant,SLC19A2,C:0.005137:15:KOREAN|G:0.5:3:Siberian|A:0.0000...,G,T,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...
22131,1,169481070,C>A,snv,999949244,normal,by-frequency;by-alfa,intron_variant,SLC19A2,A:0.000004:1:TOPMED|A:0.:0:ALFA,C,C,ATTGTTGTTTTGTTTTTTGAGACGAGTACTCGCGAGTCTCGCTCTC...,AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGG...


### Some Tests

In [53]:
# testing
snp_id_to_find = "1553211899"  # You can change this to any SNP ID you want to examine

# Find the row with the specified SNP ID and Clinical Significance not equal to normal
snp_row = mRNA_Ready[(mRNA_Ready['snp_id'] == snp_id_to_find) & 
                     (mRNA_Ready['clinical_significance'] != 'normal')]

if not snp_row.empty:
    # Store the generated mRNA for the SNP ID
    testing_mrna = snp_row.iloc[0]['generated_mRNA']
    print(f"Found mRNA for SNP ID: {snp_id_to_find} with non-normal clinical significance")
else:
    print(f"SNP ID {snp_id_to_find} with non-normal clinical significance not found in the dataset")
    # Fallback to first row if not found
    testing_mrna = mRNA_Ready.iloc[0]['generated_mRNA']
print(testing_mrna)

# Compare testing_mrna with mRNA_WT
for i in range(len(testing_mrna)):
    if testing_mrna[i] != mRNA_WT[i]:
        print(i, f' the wt is {mRNA_WT[i]}', f' the mutated is {testing_mrna[i]}')


Found mRNA for SNP ID: 1553211899 with non-normal clinical significance
AGAGGCGUCUGUAGGGUAAAGCUGGGGGUUCUGUAGCCGGAGGCGGCGGCGAGUCCAGAACGUCCUGGCCUUACAGGGAGAAGGCGUCACUCGCGGUUACAAGUGCCUGACCCUCACUCCAGUUGGCGGAGGAGGAGAAGGAAGGGGCCGGGCCGGGUCCCCUCCCCUCGCGCCCCGGAUGGAUGUGCCCGGCCCGGUGUCUCGGCGGGCGGCGGCGGCGGCGGCCACUGUGCUCCUGCGGACCGCUCGGGUCCGUCGCGAAUGCUGGUUCUUGCCGACCGCGCUGCUCUGCGCCUACGGCUUCUUCGCCAGCCUCAGGCCGUCCGAGCCCUUCCUGACCCCGUACCUGCUGGGGCCGGACAAGAACCUGACCGAGAGGGAGGUCUUCAAUGAAAUUUAUCCAGUAUGGACUUACUCUUACCUGGUGCUACUGUUUCCUGUGUUCCUUGCCACAGACUACCUCCGUUAUAAACCUGUUGUUCUACUGCAGGGGCUCAGCCUUAUUGUUACAUGGUUUAUGCUGCUCUAUGCCCAGGGACUGCUGGCCAUUCAAUUUCUAGAAUUUUUUUAUGGCAUCGCCACAGCCACUGAAAUUGCCUAUUACUCUUAUAUCUACAGUGUGGUGGACCUGGGCAUGUACCAGAAAGUCACAAGUUACUGUCGAAGUGCCACUUUGGUGGGCUUUACAGUGGGCUCUGUCCUAGGGCAAAUCCUUGUCUCAGUGGCAGGCUGGUCGCUGUUCAGCCUGAAUGUCAUCUCUCUUACCUGUGUUUCAGUGGCUUUUGCUGUGGCCUGGUUUUUACCUAUGCCACAGAAGAGCCUCUUCUUUCACCACAUUCCUUCUACCUGCCAGAGAGUGAAUGGCAUCAAGGUACAAAAUGGUGGCAUUGUUACUGACACCCCAGCUUCUAACCACCUUCCUGGCUGG

### Converting mRNA to Protien Sequnces

In [55]:

import re
from warnings import filterwarnings



aug_search_activations_count = 0
# early_stop_count will be calculated after translation

# --- 2. Variables assumed loaded (Ensure mrna_acc and mrna_df are defined) ---
# Example definitions (replace with your actual variables)
mrna_acc = "NM_006996.3" #SLC19A2
mrna_df = mRNA_Ready.copy()

# --- 3. Fetch Reference CDS Info (No changes needed here) ---
print(f"\n--- Fetching GenBank record for {mrna_acc} to find reference CDS info ---")
cds_start = None
cds_end = None
wt_protein_sequence_ref = None

if mrna_acc:
    try:
        time.sleep(entrez_delay)
        handle_gb = Entrez.efetch(db="nucleotide", id=mrna_acc, rettype="gb", retmode="text")
        record = SeqIO.read(handle_gb, "genbank")
        handle_gb.close()
        print(f"Fetched GenBank record for {mrna_acc}.")
        cds_feature_found = False
        for feature in record.features:
            if feature.type == "CDS":
                cds_start = int(feature.location.start)
                cds_end = int(feature.location.end)
                print(f"  Found reference CDS feature: Start={cds_start}, End={cds_end}")
                if "translation" in feature.qualifiers:
                    wt_protein_sequence_ref = feature.qualifiers['translation'][0]
                    print(f"  Found reference protein sequence (Length: {len(wt_protein_sequence_ref)}).")
                cds_feature_found = True
                break
        if not cds_feature_found: print(f"  WARNING: No CDS feature found for {mrna_acc}.")
        elif cds_start is None or cds_end is None: print("  WARNING: Could not determine CDS start/end.")
    except Exception as e:
        print(f"  ERROR fetching or parsing GenBank record for CDS info: {e}")
else:
    print("  WARNING: No reference mRNA Accession (`mrna_acc`) provided.")


# --- 4. Define Translation Function (Revised Logic) ---
def translate_mrna_strict_aug(mrna_sequence,
                              cds_start_index=0,
                              cds_end_index=None,
                              find_alternative_aug=True, # Changed flag name for clarity
                              search_radius=50):
    """
    Translates mRNA only if an 'AUG' start is identified.
    Priority:
    1. 'AUG' at cds_start_index.
    2. If not found & find_alternative_aug=True: Nearest 'AUG' in radius.
    3. If not found & find_alternative_aug=True: First 'AUG' after cds_start_index.
    If no 'AUG' found by these rules, returns error. Stops at first stop codon.

    Updates global counter 'aug_search_activations_count'.
    """
    global aug_search_activations_count # Allow modification of global counter

    if not isinstance(mrna_sequence, str) or mrna_sequence.startswith("Error:") or not mrna_sequence:
        return "Error: Invalid or missing mRNA sequence"

    mrna_upper = mrna_sequence.upper()
    seq_len = len(mrna_upper)

    # Use sequence length if cds_end_index is not provided or invalid
    if cds_end_index is None or cds_end_index > seq_len:
        cds_end_index_eff = seq_len
    else:
        cds_end_index_eff = cds_end_index

    original_start_index = cds_start_index if cds_start_index is not None else 0
    effective_cds_start = None # Reset: must find valid AUG
    search_activated_here = False

    # --- Step 1: Check original start index ---
    if cds_start_index is not None and cds_start_index + 3 <= seq_len:
        if mrna_upper[cds_start_index : cds_start_index + 3] == 'AUG':
            effective_cds_start = cds_start_index
            # print(f"      Info: Found 'AUG' at expected start index {cds_start_index}.") # Optional

    # --- Step 2 & 3: Search if Step 1 failed and search is enabled ---
    if effective_cds_start is None and find_alternative_aug and cds_start_index is not None:
        search_activated_here = True
        aug_search_activations_count += 1 # Increment global counter
        print(f"      Info: AUG not found at start index {original_start_index}, starting search...")

        # --- Step 2: Search within radius ---
        search_start_radius = max(0, original_start_index - search_radius)
        search_end_radius = min(seq_len, original_start_index + search_radius + 3)
        search_window_str = mrna_upper[search_start_radius:search_end_radius]

        found_aug_indices_radius = []
        current_pos = 0
        while True:
            found_pos = search_window_str.find('AUG', current_pos)
            if found_pos == -1: break
            absolute_index = search_start_radius + found_pos
            if absolute_index < cds_end_index_eff - 2:
                 found_aug_indices_radius.append(absolute_index)
            current_pos = found_pos + 1

        if found_aug_indices_radius:
            nearest_aug_index = min(found_aug_indices_radius, key=lambda idx: abs(idx - original_start_index))
            effective_cds_start = nearest_aug_index
            print(f"      --> Found AUG within radius ({search_radius}bp) at index: {effective_cds_start}")

        else:
            # --- Step 3: Search downstream (if radius search failed) ---
            print(f"      Info: No AUG found within radius. Searching downstream after index {original_start_index}...")
            # Start search *after* the original index
            downstream_search_start = original_start_index + 1
            next_aug_index = mrna_upper.find('AUG', downstream_search_start)

            if next_aug_index != -1: # Found one downstream
                 if next_aug_index < cds_end_index_eff - 2: # Check if valid
                     effective_cds_start = next_aug_index
                     print(f"      --> Found AUG downstream at index: {effective_cds_start}")
                 else:
                     print(f"      Warning: Downstream AUG ({next_aug_index}) too close to CDS end ({cds_end_index_eff}). No valid start found.")
            else:
                 print(f"      Warning: No AUG found downstream either.")

    # --- End of Search Logic ---

    # --- Step 4: Check if a valid start was found ---
    if effective_cds_start is None:
        if search_activated_here:
             # If search was activated but failed finding anything
             return "Error: No AUG start codon found after search."
        else:
             # If search wasn't activated (either disabled or AUG wasn't at start)
             if find_alternative_aug and cds_start_index is not None:
                 # This case means AUG wasn't at start, and search was enabled but failed silently earlier (shouldn't happen with current logic, but safe catch)
                 return f"Error: AUG not found at start index {original_start_index}, and no alternative found."
             else:
                 # Search wasn't enabled or start index was invalid
                 return f"Error: No AUG at provided start index {original_start_index} (search disabled or index invalid)."


    # --- Step 5: Proceed with Translation if start found ---
    # Validation already implicitly done by finding AUG < cds_end_index_eff - 2
    if effective_cds_start >= cds_end_index_eff: # Should not happen if logic above is correct
         return f"Error: Logic Error! Effective start {effective_cds_start} >= end {cds_end_index_eff}."

    cds_sequence_str = mrna_upper[effective_cds_start:cds_end_index_eff]

    if len(cds_sequence_str) < 3:
         return f"Error: Resulting CDS sequence (from {effective_cds_start} to {cds_end_index_eff}) is too short (< 3 bases)."
    if not re.fullmatch(r'[ACGUN]*', cds_sequence_str):
         return f"Error: Non-standard bases in CDS region {effective_cds_start}-{cds_end_index_eff}."

    try:
        seq_obj = Seq(cds_sequence_str)
        protein_seq = str(seq_obj.translate(to_stop=True)) # Includes '*' and stops
        return protein_seq
    except Exception as e:
        print(f"       ERROR during Biopython translation (start={effective_cds_start}): {e}")
        return f"Error: Translation failed ({e})"


# --- 5. Apply Translation and Calculate Counts ---
if mrna_df is not None:
    print(f"\n--- Translating sequences (Strict AUG start required) ---")

    # **CUSTOMIZE** Set find_alternative_aug=True to enable search if AUG not at start.
    # **CUSTOMIZE** Adjust search_radius if needed.
    FIND_ALT_AUG_ENABLED = True
    AUG_SEARCH_RADIUS = 15 # Radius for the initial search

    print(f"Find alternative AUG if not at start index: {'Enabled' if FIND_ALT_AUG_ENABLED else 'Disabled'}")
    if FIND_ALT_AUG_ENABLED: print(f"AUG Search Radius: +/- {AUG_SEARCH_RADIUS} bases")

    # Apply the strict translation function
    mrna_df['protein_sequence'] = mrna_df['generated_mRNA'].apply(
        lambda seq: translate_mrna_strict_aug(
            seq,
            cds_start_index=cds_start, # Reference start hint
            cds_end_index=cds_end,     # Reference end
            find_alternative_aug=FIND_ALT_AUG_ENABLED,
            search_radius=AUG_SEARCH_RADIUS
        )
    )

    print("\nTranslation attempt complete.")

    # --- Calculate Early Stop Count ---
    # Count rows where translation was successful (not an error) AND contains '*'
    successful_translations = mrna_df[~mrna_df['protein_sequence'].str.startswith("Error:", na=True)]
    early_stop_count = successful_translations['protein_sequence'].str.contains('\*', na=False).sum()
    successful_count = len(successful_translations)
    error_count = len(mrna_df) - successful_count

    print("\n--- Translation Summary ---")
    print(f"Total sequences processed: {len(mrna_df)}")
    print(f"Successfully translated (found AUG): {successful_count}")
    print(f"Failed translation (e.g., no AUG found): {error_count}")
    print(f"AUG search function activated: {aug_search_activations_count} times")
    print(f"Translations resulting in early stop codons ('*'): {early_stop_count}")


    print("\n--- Output DataFrame Head ---")
    with pd.option_context('display.max_rows', 10, 'display.max_columns', None, 'display.width', 1000):
        print(mrna_df.head(10)) # Show more rows to see potential errors


    # --- 6. Optional: WT Comparison (No changes needed here) ---
    # ... (Keep the comparison code from the previous version if desired) ...


    # --- 7. Save Updated DataFrame ---
    # **CUSTOMIZE** the output file path
    output_protein_csv_path = "/home/youss/DNA-bert/SLC19A3_SNP_PROTEIN_SEQUENCES_StrictAUG.csv" # Example path
    try:
        mrna_df.to_csv(output_protein_csv_path, index=False)
        print(f"\nSuccessfully saved the DataFrame with protein sequences to: {output_protein_csv_path}")
    except Exception as e:
        print(f"\nERROR saving DataFrame to CSV: {e}")

else:
    print("\n--- Skipping translation and saving because the input DataFrame was not loaded successfully. ---")

print("\n--- End of Script ---")


--- Fetching GenBank record for NM_006996.3 to find reference CDS info ---
Fetched GenBank record for NM_006996.3.
  Found reference CDS feature: Start=178, End=1672
  Found reference protein sequence (Length: 497).

--- Translating sequences (Strict AUG start required) ---
Find alternative AUG if not at start index: Enabled
AUG Search Radius: +/- 15 bases
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 177
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 183


/home/joe/miniconda3/envs/dnabert/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 179
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 176
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 182
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 177
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 177
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 183
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 179
      Info: AUG not found at start index 178, starting search...
      --> Found AUG within radius (15bp) at index: 173
      Info: AUG not found at start index

### Cleaning the Labels

In [ ]:
# Create a copy of mrna_df if not already done
final = mrna_df.copy()

# First, clean up extra whitespace
final["clinical_significance"] = final["clinical_significance"].str.strip()

# Define a mapping dictionary for the specific replacements
replacement_mapping = {
    "likely-benign;benign-likely-benign": "likely-benign",
    "likely-pathogenic;pathogenic": "likely-pathogenic",
    "uncertain-significance;likely-pathogenic": "likely-pathogenic",
    "likely-pathogenic;uncertain-significance": "likely-pathogenic",
    "likely-benign;uncertain-significance": "likely-benign",
    "likely-benign;conflicting-interpretations-of-pathogenicity": "likely-benign",
    "pathogenic;pathogenic-likely-pathogenic": "pathogenic",
    "conflicting-interpretations-of-pathogenicity": "uncertain-significance",
    "uncertain-significance;likely-benign": "likely-benign",
    "conflicting-interpretations-of-pathogenicity;uncertain-significance": "uncertain-significance",
    "conflicting-interpretations-of-pathogenicity;benign-likely-benign": "likely-benign",
    "uncertain-significance;likely-benign;conflicting-interpretations-of-pathogenicity;benign": "likely-benign",
    "conflicting-interpretations-of-pathogenicity;likely-benign": "likely-benign",
    "benign-likely-benign;conflicting-interpretations-of-pathogenicity;uncertain-significance": "likely-benign",
    "benign-likely-benign;benign": "likely-benign",
    "benign;likely-benign": "likely-benign",
    "benign;likely-benign;conflicting-interpretations-of-pathogenicity": "likely-benign",
    "benign;likely-benign;conflicting-interpretations-of-pathogenicity;uncertain-significance": "likely-benign",
    "benign;likely-benign;conflicting-interpretations-of-pathogenicity;benign-likely-benign": "likely-benign",
    "benign;likely-benign;conflicting-interpretations-of-pathogenicity;benign": "likely-benign",
    "benign;likely-benign;conflicting-interpretations-of-pathogenicity;benign-likely-benign": "likely-benign",
    "uncertain-significance;pathogenic": "pathogenic",
    "likely-pathogenic;pathogenic-likely-pathogenic": "likely-pathogenic",
    "likely-pathogenic;benign": "uncertain-significance",
    "uncertain-significance;not-provided": "uncertain-significance",
    "uncertain-significance;benign-likely-benign": "likely-benign",
    "likely-benign;uncertain-significance;benign": "likely-benign",
    "benign-likely-benign;uncertain-significance;benign": "likely-benign",
    "pathogenic-likely-pathogenic;likely-pathogenic": "likely-pathogenic",
    "benign;uncertain-significance;likely-benign": "likely-benign",
    "benign;likely-benign;uncertain-significance": "likely-benign",
    "conflicting-interpretations-of-pathogenicity;uncertain-significance;likely-benign": "likely-benign",
    "pathogenic;likely-pathogenic": "likely-pathogenic",
    "uncertain-significance;not-provided": "uncertain-significance",
    "likely-benign;benign": "likely-benign",
    "uncertain-significance;benign-likely-benign": "likely-benign",
    "uncertain-significance;benign-likely-benign": "likely-benign",
    "conflicting-interpretations-of-pathogenicity;uncertain-significance;likely-benign": "likely-benign",
    "pathogenic;uncertain-significance": "likely-pathogenic",
    "benign;uncertain-significance": "likely-benign",
    "uncertain-significance;likely-benign;benign": "likely-benign",
    

}

# Make a copy of the original column to compare later
final["clinical_significance_before"] = final["clinical_significance"].copy()

# Replace exact string matches using the mapping
final["clinical_significance"] = final["clinical_significance"].replace(replacement_mapping)

# Count how many changes were made
changes_count = (final["clinical_significance"] != final["clinical_significance_before"]).sum()
print(f"\nNumber of clinical significance labels changed: {changes_count}")

# Drop the temporary column used for comparison
final.drop("clinical_significance_before", axis=1, inplace=True)




Number of clinical significance labels changed: 12068


In [57]:
output_csv = f"final_dataset/final_ready{timestamp}.csv"
final.to_csv(output_csv, index=False)
output_pickle_path = f"final_dataset/final_ready{timestamp}.pkl"
final.to_pickle(output_pickle_path)
print(f"\nSuccessfully saved the DataFrame as pickle to: {output_pickle_path}")



Successfully saved the DataFrame as pickle to: final_dataset/final_ready20250505_171511.pkl


### Some Tests

In [58]:
# Print the first 50 proteins and their lengths
print("First 50 proteins and their lengths:")
for i, protein in enumerate(mrna_df['protein_sequence'].head(50)):
    if isinstance(protein, str):
        if protein.startswith("Error:"):
            print(f"{i+1}. {protein}")
        else:
            print(f"{i+1}. Length: {len(protein)}, Sequence: {protein[:20]}...")
    else:
        print(f"{i+1}. Invalid protein type: {type(protein)}")

# Count proteins by length
protein_lengths = mrna_df['protein_sequence'].apply(
    lambda x: len(x) if isinstance(x, str) and not x.startswith("Error:") else 0
)
print("\nProtein length statistics:")
print(f"Min length: {protein_lengths[protein_lengths > 0].min()}")
print(f"Max length: {protein_lengths.max()}")
print(f"Mean length: {protein_lengths[protein_lengths > 0].mean():.2f}")

# Check how many proteins are shorter than 496 amino acids
shorter_than_496 = protein_lengths[(protein_lengths > 0) & (protein_lengths < 496)].count()
total_valid_proteins = protein_lengths[protein_lengths > 0].count()
print(f"\nProteins shorter than 496 amino acids: {shorter_than_496} out of {total_valid_proteins} ({shorter_than_496/total_valid_proteins*100:.2f}%)")

First 50 proteins and their lengths:
1. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
2. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
3. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
4. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
5. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
6. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
7. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
8. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
9. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
10. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
11. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
12. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
13. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
14. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
15. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
16. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
17. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
18. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
19. Length: 497, Sequence: MDVPGPVSRRAAAAAATVLL...
20.

In [62]:
# Check for errors in all cells of the mrna_df DataFrame by going row by row and column by column

# Define error keywords to search for
error_keywords = ['error', 'fail', 'failed', 'failure', 'exception']

# Initialize counters and storage for errors
total_errors = 0
errors_by_column = {}
error_examples = {}

print("\n--- Checking for errors in the mrna_df DataFrame (row by row, column by column) ---")
print(f"Total rows to check: {len(final)}")
print(f"Total columns to check: {len(final.columns)}")

# Initialize error counters for each column
for col in final.columns:
    errors_by_column[col] = 0
    error_examples[col] = []

# Go through each row and column to check for errors
for idx, row in final.iterrows():
    if idx % 5000 == 0 and idx > 0:
        print(f"Processed {idx} rows...")
    
    for col in final.columns:
        cell_value = row[col]
        
        # Check if the cell contains any error keywords
        if isinstance(cell_value, str):
            cell_lower = cell_value.lower()
            if any(keyword in cell_lower for keyword in error_keywords):
                total_errors += 1
                errors_by_column[col] += 1
                
                # Store up to 3 examples per column
                if len(error_examples[col]) < 3:
                    # Truncate long error messages for display
                    display_value = cell_value[:100] + "..." if len(cell_value) > 100 else cell_value
                    error_examples[col].append((idx, display_value))

print(f"\nFound {total_errors} total cells containing error keywords")

# Display results by column
print("\nErrors by column:")
for col, count in errors_by_column.items():
    if count > 0:
        percentage = (count / len(final)) * 100
        print(f"  {col}: {count} errors ({percentage:.2f}%)")
        
        # Show examples
        print("  Examples:")
        for row_idx, example in error_examples[col]:
            print(f"    Row {row_idx}: {example}")

# If no errors found
if total_errors == 0:
    print("\nNo cells containing error keywords were found in the DataFrame")

# Additional checks for NaN and None values
print("\n--- Additional checks ---")

# Check for NaN values
nan_counts = final.isna().sum()
columns_with_nans = nan_counts[nan_counts > 0]

if len(columns_with_nans) > 0:
    print("\nColumns with NaN values:")
    for col, count in columns_with_nans.items():
        print(f"  {col}: {count} NaN values ({count/len(final)*100:.2f}%)")
else:
    print("\nNo NaN values found in any column")

# Check for None values
none_counts = final.applymap(lambda x: x is None).sum()
columns_with_nones = none_counts[none_counts > 0]

if len(columns_with_nones) > 0:
    print("\nColumns with None values:")
    for col, count in columns_with_nones.items():
        print(f"  {col}: {count} None values ({count/len(final)*100:.2f}%)")
else:
    print("\nNo None values found in any column")

# Summary
print(f"\nDataFrame shape: {final.shape} (rows, columns)")
print(f"Total cells: {final.size}")
print(f"Total cells with issues: {total_errors + nan_counts.sum() + none_counts.sum()}")





--- Checking for errors in the mrna_df DataFrame (row by row, column by column) ---
Total rows to check: 22133
Total columns to check: 15
Processed 5000 rows...
Processed 10000 rows...
Processed 15000 rows...
Processed 20000 rows...

Found 0 total cells containing error keywords

Errors by column:

No cells containing error keywords were found in the DataFrame

--- Additional checks ---

Columns with NaN values:
  clinical_significance: 12015 NaN values (54.29%)
  validation_status: 921 NaN values (4.16%)
  function_class: 2 NaN values (0.01%)
  frequency: 1352 NaN values (6.11%)

No None values found in any column

DataFrame shape: (22133, 15) (rows, columns)
Total cells: 331995
Total cells with issues: 14290


/tmp/ipykernel_1475652/3046885013.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  none_counts = final.applymap(lambda x: x is None).sum()
